In [3]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.cluster.util import cosine_distance
from nltk import sent_tokenize,word_tokenize
import numpy as np
import re
import numpy
from numpy import delete
import math
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
def read_article(para):
    file = open( r"C:\Users\user\vijay.txt",encoding='utf-8')
    filedata = file.read()
#     print(filedata)
    sentences = sent_tokenize(filedata)
    #print("Tokenized Sentences: ",sentences)
    sent=word_tokenize(filedata)
#     print(sentences)
    return sentences



def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    #ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        #print(words)
        for word in words:
            word = word.lower()
            #word = ps.stem(word)
            #print(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence
         
        tf_matrix[sent] = tf_table

    return tf_matrix


def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))
            
        idf_matrix[sent] = idf_table

    return idf_matrix

def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}
    sum=0
    text=[]
    

    
    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)
            
        
            #print(tf_idf_table[word1])
        tf_idf_matrix[sent1]= tf_idf_table
#         print(tf_idf_matrix[sent1])
        
    return tf_idf_matrix
def _tf_idf(tf_matrix, idf_matrix):
    tf_idf_matrix = {}
    sum=0
    text=[]
    

    
    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)
            
        
            #print(tf_idf_table[word1])
            tf_idf_matrix[sent1]= tf_idf_table
#         print(tf_idf_matrix[sent1])
        s=len(tf_idf_matrix[sent1])
#         print(s)
        for i in tf_idf_matrix[sent1].values():
            
            sum=(sum+i)
            
        text.append(sum/s)
        sum=0 
    i=23  
    print(text[i])
    return text
def cosine_value(sentences):
    count_vectorizer = CountVectorizer(stop_words='english')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(sentences)


# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=[sentences])
    df
    cosine=cosine_similarity(df, df)
#     print(cosine)
    return cosine
def cal_pop_fitness(text, pop):
    n=[]
    arr=[]
    arr1=[]
    final=[]
    final1=[]
    score=[]
    file = open( r"C:\Users\user\vijay.txt",encoding='utf-8')
    filedata = file.read()
#     print(filedata)
    sentences = sent_tokenize(filedata)
    cosine=cosine_value(sentences)
    x=np.array(cosine)
    y=np.argwhere(x>0.8)
#     z=np.unique(y)

    k=0
    for i in range(len(y)): 
        if(y[i][k] ==y[i][k+1]): 
            continue

        elif(text[y[i][k]]>text[y[i][k+1]]):
            n.append(text[y[i][k]])
            arr.append(y[i][k])
            arr.append(y[i][k+1])
          
        else:
            n.append(text[y[i][k+1]])
#     print(np.unique(n))#returning the greater score
    arr1=np.unique(arr)
#     print(arr1)
#     print(y)
#     print(arr1)#returning the comparing sentences
    for i in range(len(y)):
        if (y[i][k] not in arr1):
            final.append(y[i][k])
#     print(final)#returning the other sentences
    final1=np.unique(final)
#     print(final1)
#     print(len(final1))
    for i in range(len(final1)-1):
        score.append(text[final1[i]])
#     print(score)# returning the other sentence score
    final_list=[n+score]
    final_score=np.unique(final_list)
    print("final_score  ",final_score)
    return final_score
def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = numpy.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover


def generate_summary(para, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(para)
    
    total_documents = len(sentences)
    
    freq_matrix = _create_frequency_matrix(sentences)
    #print("freqency_matrix",freq_matrix)

    
    tf_matrix = _create_tf_matrix(freq_matrix)
    #print("tf_matrix:",tf_matrix)
    count_doc_per_words = _create_documents_per_words(freq_matrix)
    
    idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    #print("idf_matrix",idf_matrix)

    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    #print("\n\n")
    #print("tf-idf: ",tf_idf_matrix)


    
# Create the Document Term Matrix
    
#genetic optimizer part begin
    text=_tf_idf(tf_matrix, idf_matrix)
    num_weights=6
    sol_per_pop = 30
    num_parents_mating = 2
    # Defining the population size.
    pop_size = (sol_per_pop,num_weights) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.
#Creating the initial population.
    new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)
    print(new_population)
    num_generations = 5
    for generation in range(num_generations):
        print("Generation : ", generation)
    # Measing the fitness of each chromosome in the population.
        fitness = cal_pop_fitness(text, new_population)
        # Selecting the best parents in the population for mating.
        parents = select_mating_pool(new_population, fitness, num_parents_mating)

    # Generating next generation using crossover.
        offspring_crossover = crossover(parents,
                                       offspring_size=(pop_size[0]-parents.shape[0], num_weights))

    # Adding some variations to the offsrping using mutation.
        offspring_mutation = mutation(offspring_crossover)

    # Creating the new population based on the parents and offspring.
        new_population[0:parents.shape[0], :] = parents
        new_population[parents.shape[0]:, :] = offspring_mutation

    # The best result in the current iteration.
        print("Best result : ", numpy.max(fitness))#numpy.sum(new_population*text, axis=1)))

# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
#     text=_tf_idf(tf_matrix, idf_matrix)
    fitness = cal_pop_fitness(text, new_population)
# Then return the index of that solution corresponding to the best fitness.
    best_match_idx = numpy.where(fitness == numpy.max(fitness))

    print("Best solution : ", new_population[best_match_idx, :])
    print("Best solution fitness : ", fitness[best_match_idx])


generate_summary( "msft.txt", 2)
#Genetic algorithm part

0.06792992745623491
[[ 6.24303900e-01 -2.34162549e+00 -1.65453658e+00  3.77760547e+00
   3.14392455e+00  7.84839918e-02]
 [-2.75771015e+00 -3.15270143e+00  1.12691524e+00 -3.33850444e+00
  -2.27456189e+00 -4.53237743e-01]
 [ 2.62105084e+00 -1.30483092e+00  1.75191055e+00  3.22065227e-02
   2.04069530e+00  2.75438578e+00]
 [-3.64771425e+00 -1.78413460e+00  3.12386019e+00  3.96212567e+00
   2.53409635e+00  1.68511959e+00]
 [ 1.46812712e-01  1.72986956e-01  3.93743344e+00 -1.50242956e+00
  -2.77035764e+00 -3.36084382e+00]
 [ 2.82608864e+00  3.77558898e+00 -1.22010709e+00  3.96490698e+00
  -1.38437795e+00  2.01993265e+00]
 [ 1.52357519e+00 -4.83152205e-01  1.03133173e+00  2.56195717e+00
   3.17300550e+00  3.88307084e+00]
 [ 2.26906961e+00  8.73358530e-01 -1.84389152e+00 -2.73740613e+00
  -8.33934012e-01  3.43891190e+00]
 [ 1.28556718e+00  1.17322768e+00 -2.59054967e+00  8.63743593e-01
   3.94403884e+00 -3.52885320e+00]
 [-1.89899623e+00 -3.89109176e+00  1.01421464e+00 -1.90808399e-01
  -6.